In [ ]:
%run includes/shared-setup.ipynb

In [ ]:
def graph_network_peerings(network_peerings_df):

    # Create a graph from the DataFrame
    G = nx.from_pandas_edgelist(network_peerings_df, 'name', 'peering_network')

    # Generate positions of the nodes
    pos = nx.spring_layout(G)

    # Extract node and edge information
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    node_x = [pos[k][0] for k in G.nodes()]
    node_y = [pos[k][1] for k in G.nodes()]

    # Create the edge trace
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    # Create the node trace with hover information
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=False,
            size=10,
            line=dict(width=2)),
        text=list(G.nodes()),
        textposition="top right",
        hovertext=[f'Name: {node}<br>Connections: {G.degree[node]}' for node in G.nodes()])

    # Create the network graph
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=0),
                        plot_bgcolor='white',
                        paper_bgcolor='white',
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

    # Show plot
    fig.show()

In [ ]:
def show_network_summary_cards(networks_df, num_interconnects, num_vpns, num_external_vpn_gateways, num_firewalls, num_public_fws, num_forwarding_rules):
    # Convert the 'num_subnets' column to integers
    networks_df['num_subnets'] = networks_df['num_subnets'].astype(int, errors='ignore')  # Using 'ignore' will leave non-convertible types as is
    
    # Now get the total number of networks and subnets
    total_networks = len(networks_df)
    total_subnets = networks_df['num_subnets'].sum()

    cards_data = [
        ("Total Networks", total_networks), 
        ("Total Subnets", total_subnets),
        ("Total Interconnects", num_interconnects),
        ("VPN Gateways", num_vpns),
        ("External VPN Gateways", num_external_vpn_gateways),
        ("Firewalls", num_firewalls),
        ("Public Firewalls", num_public_fws),
        ("Forwarding Rules", num_forwarding_rules)
        ]

    # summary cards
    display_cards(cards_data)

In [ ]:
with open('/tmp/compute_enabled_projects.pkl', 'rb') as f:
    compute_enabled_projects = pickle.load(f)

In [ ]:
def generate_xpn_tree(xpn_resources_df):
    # Initialize the Tree
    tree = Tree()
    
    # Create a dictionary to hold host nodes
    host_nodes = {}
    
    # Loop through the DataFrame rows
    for index, row in xpn_resources_df.iterrows():
        host_project = row['hostProject']
        service_project = row['serviceProject']
    
        # Create the host node if it doesn't exist yet
        if host_project not in host_nodes:
            host_node = Node(host_project)
            host_nodes[host_project] = host_node
            tree.add_node(host_node)
    
        # Create and add the service node to the host node
        service_node = Node(service_project)
        host_nodes[host_project].add_node(service_node)
    
    # Display the tree
    return tree